# Assignment 7

### Part A

In [186]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [187]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [188]:
def expected_return(P: pd.DataFrame, rf, mkt_rp):
    # Write your code here
    expRetP = 0
    for i in range(len(P)):
        ret = rf + P.get("Beta")[i] * mkt_rp
        expRetP += P.get("Amount Invested")[i] / P.get("Amount Invested").sum() * ret
    return expRetP

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [189]:
def volatility(P: pd.DataFrame, cor):
    # Write your code here
    volatilityP = 0
    for i in range(len(P)):
        for j in range(len(P)):
            if i == j:
                volatilityP += (P.get("Amount Invested")[i] / P.get("Amount Invested").sum() * P.get("Volatility")[i]) ** 2
            else:
                volatilityP += P.get("Amount Invested")[i] * P.get("Amount Invested")[j] / (P.get("Amount Invested").sum() ** 2) * cor * P.get("Volatility")[i] * P.get("Volatility")[j]
    return np.sqrt(volatilityP)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [190]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    expected_return = rf + ((vol / mkt_vol) * mkt_rp)
    return expected_return


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [496]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    cusip = []
    expRet = []
    beta = []

    retPath = pd.read_csv(return_path)
    facPath = pd.read_csv(factors_path)
    facPath["Mkt-RF"] = facPath["Mkt-RF"] / 100
    facPath["RF"] = facPath["RF"] / 100
    retPath["date"] = retPath["year"] * 100 + retPath["month"]

    df = retPath.merge(facPath, how="inner", on="date")
    df.sort_values(by=["CUSIP", "date"], inplace=True)
    df["Ri - Rf"] = df["RET"] - df["RF"]

    for index, data in df.groupby('CUSIP'):
        data.reset_index(drop=True, inplace=True)
        cusip.append(data["CUSIP"][0])
        beta.append(data.drop("CUSIP", axis=1)[["Ri - Rf", "Mkt-RF"]].cov().iloc[0,1] / data["Mkt-RF"].var())
        expRet.append(data.get("RET").mean())
    output = pd.DataFrame(data = {
        "CUSIP" : cusip ,
        "EXP_RETURN" : expRet,
        "BETA" : beta,
    })
    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [506]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    return np.polyfit(data.get("BETA"), data.get("EXP_RETURN"), 1)[0]

In [507]:
slope(x)

0.004773236586034279